# Download and Import Libraries

In [1]:
!pip install cdflib

from google.colab import drive
import pandas as pd
import numpy as np
import xarray as xr
import torch
import datetime
from datetime import datetime, timedelta
import cdflib
import os
import sys

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 397 kB/s 


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Load Datasets

In [3]:
days = [1, 2, 3, 4, 5, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 28]
YEAR = 2008
MONTH = 2

prob_dir = '/content/drive/MyDrive/substorm_prediction/probabilities/fsmi/'
mag_dir = '/content/drive/MyDrive/substorm_prediction/Data_science/'

prob_mag_dfs = []

for day in days:
  date = datetime(YEAR, MONTH, day)

  # filename
  prob_file = 'probability_' + date.strftime('%Y%m%d') + '.csv'
  mag_file = 'thg_l2_mag_fsmi_' + date.strftime('%Y%m%d') + '_v01.cdf'

  # read probability
  prob_df = pd.read_csv(prob_dir + prob_file)
  prob_df['time'] = pd.to_datetime(prob_df['time'])

  # read magnetometer and convert to DataFrame
  mag_xr = cdflib.cdf_to_xarray(mag_dir + mag_file)
  mag_df = pd.DataFrame(data=mag_xr['thg_mag_fsmi'].to_numpy(), columns=['H_North', 'E_East', 'Z_Down'])
  mag_df['time'] = np.arange(date, date+timedelta(days=1), timedelta(seconds=0.5))
  mag_df = mag_df.iloc[::2, :]

  # merge probability and magnetometer
  combined_df = prob_df.merge(mag_df, on='time')
  combined_df = combined_df.reset_index()

  prob_mag_dfs.apbpend(combined_df)
  

ISTP Compliance Warning: thg_mag_fsmi_epoch is listed as the DEPEND_0 for variable thg_mag_fsmi, but the dimensions do not match.
ISTP Compliance Warning: thg_mag_fsmi_epoch is listed as the DEPEND_0 for variable thg_mag_fsmi, but the dimensions do not match.
ISTP Compliance Warning: thg_mag_fsmi_epoch is listed as the DEPEND_0 for variable thg_mag_fsmi, but the dimensions do not match.
ISTP Compliance Warning: thg_mag_fsmi_epoch is listed as the DEPEND_0 for variable thg_mag_fsmi, but the dimensions do not match.
ISTP Compliance Warning: thg_mag_fsmi_epoch is listed as the DEPEND_0 for variable thg_mag_fsmi, but the dimensions do not match.
ISTP Compliance Warning: thg_mag_fsmi_epoch is listed as the DEPEND_0 for variable thg_mag_fsmi, but the dimensions do not match.
ISTP Compliance Warning: thg_mag_fsmi_epoch is listed as the DEPEND_0 for variable thg_mag_fsmi, but the dimensions do not match.
ISTP Compliance Warning: thg_mag_fsmi_epoch is listed as the DEPEND_0 for variable thg_mag

# Load Substorm Onsets

In [4]:
onsets = []

with open('/content/drive/MyDrive/substorm_prediction/substorm_onsets.txt', 'r') as f:
  for line in f.readlines():
    tokens = line.split(' ')
    if tokens[1].strip('\n') == 'nothing':
      continue

    date = datetime.strptime(tokens[0], '%Y-%m-%d')
    for time in tokens[1:]:
      onsets.append(date + timedelta(hours=int(time[:2]), minutes=int(time[3:5])))
  
for onset in onsets:
  print(onset)

2008-02-01 06:14:00
2008-02-01 10:08:00
2008-02-01 11:12:00
2008-02-02 03:41:00
2008-02-02 07:40:00
2008-02-02 08:12:00
2008-02-02 12:42:00
2008-02-03 04:56:00
2008-02-03 07:15:00
2008-02-03 08:01:00
2008-02-03 09:10:00
2008-02-04 04:02:00
2008-02-04 08:43:00
2008-02-05 04:36:00
2008-02-05 12:11:00
2008-02-08 06:27:00
2008-02-10 05:50:00
2008-02-10 07:16:00
2008-02-10 10:25:00
2008-02-10 12:24:00
2008-02-11 04:27:00
2008-02-11 07:15:00
2008-02-11 08:31:00
2008-02-12 06:27:00
2008-02-12 08:49:00
2008-02-12 09:42:00
2008-02-13 07:53:00
2008-02-14 06:56:00
2008-02-15 06:13:00
2008-02-15 10:14:00
2008-02-16 05:10:00
2008-02-16 08:06:00
2008-02-17 07:32:00
2008-02-17 08:17:00
2008-02-17 09:22:00
2008-02-18 09:15:00
2008-02-18 11:52:00
2008-02-19 05:24:00
2008-02-19 07:24:00
2008-02-19 08:12:00
2008-02-20 07:06:00
2008-02-21 08:15:00
2008-02-21 12:09:00
2008-02-22 06:05:00
2008-02-22 06:48:00
2008-02-22 09:28:00
2008-02-23 06:21:00
2008-02-23 07:13:00
2008-02-24 10:02:00
2008-02-25 08:14:00


# Normalization

In [5]:
# features
columns = ['arc', 'diffuse', 'discrete', 'cloudy', 'moon', 'clear', 
           'H_North',	'E_East',	'Z_Down']

# concatenate and normalization
big_df = pd.concat(prob_mag_dfs)
big_df[columns] = ((big_df[columns] - big_df[columns].mean()) / big_df[columns].std()).astype(float)

# split by day
normalized_dfs = []
for day in days:
  day_df = big_df[big_df['time'].dt.day == day]
  normalized_dfs.append(day_df)

# Split to 30-minutes Datapoints

In [ ]:
# split data
inputs = []
labels = []

for df in normalized_dfs:
  # start at minutes = 0, 15, 30, 60
  raw_start, raw_end = df['time'][0], df['time'][len(df)-1]
  t1 = datetime(raw_start.year, raw_start.month, raw_start.day, raw_start.hour)
  for start_t in np.arange(t1, t1+timedelta(hours=1, minutes=1), timedelta(minutes=15)):
    if raw_start <= start_t:
      new_start = start_t
      break
  t2 = datetime(raw_end.year, raw_end.month, raw_end.day, raw_end.hour) - timedelta(seconds=3)
  for end_t in np.arange(t2, t2+timedelta(minutes=46), timedelta(minutes=15)):
    if raw_end < end_t:
      break
    new_end = end_t

  # split into 30-minutes chunks (there are overlaps due to 15-min shift)
  for start_time in np.arange(new_start, new_end, np.timedelta64(15, 'm')):
      # split dataset
      end_time = start_time + np.timedelta64(30, 'm')
      datapoint = df[(df['time'] >= start_time) & (df['time'] < end_time)]
      datapoint = datapoint[columns]
      
      # discard missing values
      if datapoint.shape[0] != 20 * 30:
        continue

      # label 1 if substorms happens during the 30 minutes
      is_substorm = 0
      for onset in onsets:
        if start_time <= onset and end_time > onset:
          is_substorm = 1
          break

      #print(datapoint.shape)
      inputs.append(torch.Tensor(datapoint.to_numpy()))
      labels.append(float(is_substorm))

# convert to tensor
inputs = torch.stack(inputs)
labels = torch.tensor(labels)

# split to train and test
train_input = inputs[:int(0.8 * inputs.size()[0])]
train_label = labels[:int(0.8 * labels.size()[0])]
test_input = inputs[int(0.8 * inputs.size()[0]):]
test_label = labels[int(0.8 * labels.size()[0]):]

# Save Datasets

Inputs and labels for both training and test are saved as `torch.tensor`

In [ ]:
train_dir = '/content/drive/MyDrive/substorm_prediction/train/'
test_dir = '/content/drive/MyDrive/substorm_prediction/test/'

torch.save(train_input, train_dir + 'input.pt')
torch.save(train_label, train_dir + 'label.pt')
torch.save(test_input, test_dir + 'input.pt')
torch.save(test_label, test_dir + 'label.pt')